# Example 1

### End-to-end workflow to calculate conformer proportions and NMR spectra of strychnine with AQME from a SMILES string

This workflow includes:

i) RDKit conformer sampling \
ii) Gaussian geometry optimization (B3LYP/6-31+G(d,p)) \
iii) Fixing errors and imaginary frequencies of the output files \
iv) Calculation and analysis of NMR chemical shifts for the conformers generated \
v) GoodVibes calculation of Boltzmann distributions using Gibbs free energies at 298.15 K

#### Steps involved in this example

- Step 1: Importing AQME and other python modules
- Step 2: CSEARCH conformational sampling
- Step 3: Creating Gaussian input files for optimization and frequency with QPREP
- Step 4: Running Gaussian inputs for optimization and frequency calcs externally
- Step 5: QCORR analysis including isomerization filter
- Step 6: Resubmission of unsuccessful calculations (if any) with suggestions from AQME
- Step 7: Creating Gaussian input files for NMR calcs with QPREP
- Step 8: Running Gaussian NMR calcs
- Step 9: Obtaining Boltzmann weighted NMR shifts with QDESCP
- Step 10: Calculating conformer populations with GoodVibes

###  Step 1: Importing AQME and other python modules

In [1]:
import os, subprocess,shutil, glob
from pathlib import Path 
from aqme.csearch import csearch
from aqme.qprep import qprep
from aqme.qcorr import qcorr
from aqme.qdescp import qdescp

[13:03:31] Enabling RDKit 2019.09.3 jupyter extensions
RDKit WARNING: [13:03:31] Enabling RDKit 2019.09.3 jupyter extensions


### Step 2: CSEARCH conformational sampling

In [2]:
name = 'Strychnine'
smi = 'C1CN2CC3=CCO[C@H]4CC(=O)N5[C@H]6[C@H]4[C@H]3C[C@H]2[C@@]61C7=CC=CC=C75'
program = 'rdkit'

sdf_path = f'{os.getcwd()}/{name}_sdf-files' # folder where the SDF files are generated

csearch(destination=sdf_path,program=program,smi=smi,name=name)

AQME v 1.2 2022/09/07 13:03:31 
Citation: AQME v 1.2, Alegre-Requena, J. V.; Sowndarya, S.; Perez-Soto, R.; Alturaifi, T. M.; Paton, R. S., 2022. https://github.com/jvalegre/aqme



Starting CSEARCH with 1 job(s) (SDF, XYZ, CSV, etc. files might contain multiple jobs/structures inside)



   ----- Strychnine -----


o  Applying filters to initial conformers


Time CSEARCH: 16.34 seconds




### Step 3: Creating Gaussian input files for optimization and frequency with QPREP

In [3]:
program = 'gaussian'
qm_input = 'B3LYP/6-31+G(d,p) opt freq'
mem='24GB'
nprocs=12

sdf_rdkit_files = f'{sdf_path}/*.sdf' # SDF files from Step 2
com_path = f'{os.getcwd()}/{name}_com-files' # folder where the COM files are generated

qprep(destination=com_path,files=sdf_rdkit_files,program=program,
        qm_input=qm_input,mem=mem,nprocs=nprocs)

AQME v 1.2 2022/09/07 13:03:31 
Citation: AQME v 1.2, Alegre-Requena, J. V.; Sowndarya, S.; Perez-Soto, R.; Alturaifi, T. M.; Paton, R. S., 2022. https://github.com/jvalegre/aqme


o  Strychnine_rdkit successfully processed at /home/svss/Project-DBcg-Debug/aqme2_20220329/20220906-example-end-to-end/End-to-end_Workflows/End-to-end workflow 1, strychnine/AQME inputs/end-to-end-1-Jupyter-Notebook/Strychnine_com-files


Time QPREP: 0.01 seconds




### Step 4: Running Gaussian inputs for optimization and frequency calcs externally

In [4]:
# Run the generated COM files (in com_path, see PATH in Step 3) with Gaussian

### Step 5: QCORR analysis including isomerization filter

In [5]:
log_files=f'{com_path}/*.log' # LOG files from Step 4

qcorr(files=log_files,freq_conv='opt=(calcfc,maxstep=5)',isom_type='com',isom_inputs=com_path,nprocs=24,mem='96GB')

AQME v 1.2 2022/09/07 13:03:31 
Citation: AQME v 1.2, Alegre-Requena, J. V.; Sowndarya, S.; Perez-Soto, R.; Alturaifi, T. M.; Paton, R. S., 2022. https://github.com/jvalegre/aqme


o  Analyzing output files in /home/svss/Project-DBcg-Debug/aqme2_20220329/20220906-example-end-to-end/End-to-end_Workflows/End-to-end workflow 1, strychnine/AQME inputs/end-to-end-1-Jupyter-Notebook/Strychnine_com-files


Strychnine_rdkit_conf_1.log: Termination = normal, Error type = none

Strychnine_rdkit_conf_2.log: Termination = normal, Error type = none


-- Full check analysis --
o  Same program (Gaussian 16, Revision C.01) used in all the calculations
o  Same grid_type (ultrafine) used in all the calculations
o  Same level_of_theory (B3LYP/6-31+G(d,p)) used in all the calculations
o  Same dispersion (none) used in all the calculations
o  Same solvation (gas_phase) used in all the calculations


 Time QCORR: 8.14 seconds




### Step 6: Resubmission of unsuccessful calculations (if any) with suggestions from AQME

In [6]:
# Run the generated COM files (in fixed_inp_folder) with Gaussian

### Step 7: Creating Gaussian input files for NMR calcs with QPREP

In [7]:
program = 'gaussian'
qm_input = 'B3LYP/6-311+G(2d,p) scrf=(solvent=chloroform,smd) nmr=giao'
mem='24GB'
nprocs=12

success_folder = com_path+'/success' # folder where the successful LOG files are stored during the QCORR cycles (Steps 5 and 6)
log_files = f'{success_folder}/*.log'
sp_path = f'{os.getcwd()}/{name}_sp-files' # folder to store the new COM inputs for single point NMR calcs

qprep(w_dir_main=success_folder,destination=sp_path,files=log_files,program=program,qm_input=qm_input,mem=mem,nprocs=nprocs,suffix='SP')

AQME v 1.2 2022/09/07 13:03:31 
Citation: AQME v 1.2, Alegre-Requena, J. V.; Sowndarya, S.; Perez-Soto, R.; Alturaifi, T. M.; Paton, R. S., 2022. https://github.com/jvalegre/aqme


o  Strychnine_rdkit_conf_1 successfully processed at /home/svss/Project-DBcg-Debug/aqme2_20220329/20220906-example-end-to-end/End-to-end_Workflows/End-to-end workflow 1, strychnine/AQME inputs/end-to-end-1-Jupyter-Notebook/Strychnine_sp-files

o  Strychnine_rdkit_conf_2 successfully processed at /home/svss/Project-DBcg-Debug/aqme2_20220329/20220906-example-end-to-end/End-to-end_Workflows/End-to-end workflow 1, strychnine/AQME inputs/end-to-end-1-Jupyter-Notebook/Strychnine_sp-files


Time QPREP: 0.01 seconds




### Step 8: Running Gaussian NMR calcs

In [8]:
# Run the generated COM files (in sp_path, see PATH in Step 7) with Gaussian

### Step 9: Obtaining Boltzmann weighted NMR shifts with QDESCP

In [9]:
# Create JSON files with QCORR to store the information from the resulting LOG files
log_files=f'{sp_path}/*.log'
qcorr(files=log_files)

# Analyze the JSON files to calculate the Boltzmann averaged shielding tensors
json_folder = sp_path+'/success/SP_calcs/json_files' # folder where the JSON files were just created with QCORR
json_files=f'{json_folder}/*.json'
nmr_path = f'{os.getcwd()}/{name}_nmr-files' # folder to store the results from QDESCP

qdescp(program='nmr',boltz=True,files=json_files,destination=nmr_path,nmr_slope=[-1.0537, -1.0784],nmr_intercept=[181.7815,31.8723])

AQME v 1.2 2022/09/07 13:03:31 
Citation: AQME v 1.2, Alegre-Requena, J. V.; Sowndarya, S.; Perez-Soto, R.; Alturaifi, T. M.; Paton, R. S., 2022. https://github.com/jvalegre/aqme


o  Analyzing output files in /home/svss/Project-DBcg-Debug/aqme2_20220329/20220906-example-end-to-end/End-to-end_Workflows/End-to-end workflow 1, strychnine/AQME inputs/end-to-end-1-Jupyter-Notebook/Strychnine_sp-files


Strychnine_rdkit_conf_1_SP.log: Termination = normal, Error type = sp_calc

Strychnine_rdkit_conf_2_SP.log: Termination = normal, Error type = sp_calc


x  No normal terminations with no errors to run the full check analysis


 Time QCORR: 7.56 seconds


AQME v 1.2 2022/09/07 13:03:31 
Citation: AQME v 1.2, Alegre-Requena, J. V.; Sowndarya, S.; Perez-Soto, R.; Alturaifi, T. M.; Paton, R. S., 2022. https://github.com/jvalegre/aqme


o  QDESCP successfully done at /home/svss/Project-DBcg-Debug/aqme2_20220329/20220906-example-end-to-end/End-to-end_Workflows/End-to-end workflow 1, strychnine/AQM

### Step 10: Calculating conformer populations with GoodVibes

In [11]:
log_files = glob.glob(f'{success_folder}/*.log')

w_dir_main  = Path(os.getcwd())
GV_folder = w_dir_main.joinpath('Strychine_GoodVibes-analysis')
GV_folder.mkdir(exist_ok=True, parents=True)

for file in log_files:
	shutil.copy(file, GV_folder)

# run GoodVibes
os.chdir(GV_folder)
subprocess.run(['python', '-m', 'goodvibes', '--xyz','-c','1', '*.log','--boltz'])
os.chdir(w_dir_main)


   GoodVibes v3.2 2022/09/07 13:08:05
   Citation: Luchini, G.; Alegre-Requena, J. V.; Funes-Ardoiz, I.; Paton, R. S. F1000Research, 2020, 9, 291.
   GoodVibes version 3.2 DOI: 10.12688/f1000research.22758.1

o  Requested: --xyz -c 1 --boltz 

   Temperature = 298.15 Kelvin   Concentration = 1.0 mol/L
   All energetic values below shown in Hartree unless otherwise specified.

   Using vibrational scale factor 1.0 for B3LYP/6-31+G(d,p) level of theory

   Entropic quasi-harmonic treatment: frequency cut-off value of 100.0 wavenumbers will be applied.
   QS = Grimme: Using a mixture of RRHO and Free-rotor vibrational entropies.
   REF: Grimme, S. Chem. Eur. J. 2012, 18, 9955-9964


   Structure                                           E        ZPE             H        T.S     T.qh-S          G(T)       qh-G(T)  Boltz
   ***************************************************************************************************************************************
o  Strychnine_rdkit_conf_1       